### Topic - Using tools
##### Enable models to fetch data and take actions.

In [1]:
import json
import os

import requests
from openai import OpenAI
from pydantic import BaseModel, Field

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 🛠️ Function Calling

**Function Calling**: Lets the AI call your backend functions with structured arguments based on the user's request.

- Allows dynamic interaction between AI and external tools or APIs.
- Useful for tasks like retrieving data, triggering actions, or generating responses based on real-time inputs.


---
**Introduction:** Using a Weather API, we can model it as a tool and make it available to the LLM. Based on the context, the LLM looks at the available tools and, depending on the user's input or question, decides whether it wants to use the tool or not.

##### Define the tool (function) that we want to call


In [3]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

#### Step 1: Call model with get_weather tool defined


In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]


In [5]:
system_prompt = "You are a helpful weather assistant."

In [6]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]


In [7]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

#### Step 2: Model decides to call function(s)


In [8]:
completion.model_dump()

{'id': 'chatcmpl-BU5IEsV3bgs0WkvCmajrAaTU2JQnS',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_aUiL56QHAhLapmKMdfBlhPW5',
      'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1746509886,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_d8864f8b6b',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 66,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [9]:
tool_calls = completion.choices[0].message.tool_calls
formatted_tool_call = json.dumps(tool_calls[0].model_dump(), indent=4)
print(formatted_tool_call)


{
    "id": "call_aUiL56QHAhLapmKMdfBlhPW5",
    "function": {
        "arguments": "{\"latitude\":48.8566,\"longitude\":2.3522}",
        "name": "get_weather"
    },
    "type": "function"
}


#### Step 3: Execute get_weather function

**Important Note:** The LLM is not going to call the tool for you. It will just return the function call with the parameters that you need to pass to the function. 

In [10]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

In [11]:
for tool_call in tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [12]:
name

'get_weather'

In [13]:
args

{'latitude': 48.8566, 'longitude': 2.3522}

In [14]:
messages

[{'role': 'system', 'content': 'You are a helpful weather assistant.'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aUiL56QHAhLapmKMdfBlhPW5', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_aUiL56QHAhLapmKMdfBlhPW5',
  'content': '{"time": "2025-05-06T05:30", "interval": 900, "temperature_2m": 7.2, "wind_speed_10m": 14.5}'}]

In [15]:
result

{'time': '2025-05-06T05:30',
 'interval': 900,
 'temperature_2m': 7.2,
 'wind_speed_10m': 14.5}

#### Step 4: Supply result and call model again

In [16]:
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description="The current temperature in celsius for the given location."
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

In [17]:
completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)



#### Step 5: Check model response

In [18]:
final_response = completion_2.choices[0].message.parsed

In [19]:
final_response.model_dump()

{'temperature': 7.2,
 'response': 'The current temperature in Paris is 7.2°C with a wind speed of 14.5 km/h.'}

In [20]:
final_response.temperature

7.2

In [21]:
final_response.response

'The current temperature in Paris is 7.2°C with a wind speed of 14.5 km/h.'

> 📖 More details of Function Calling on the [OpenAI Docs](https://platform.openai.com/docs/guides/function-calling)



## LATEST

In [22]:
from openai import OpenAI

client = OpenAI()

In [23]:
tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "The recipient email address."},
                "subject": {"type": "string", "description": "Email subject line."},
                "body": {"type": "string", "description": "Body of the email message."},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

In [24]:
response = client.responses.create(
    model="gpt-4o",
    input="Can you send an email to ilan@example.com and katia@example.com saying hi?",
    tools=tools,
)

In [25]:
print(response.output)

[ResponseFunctionToolCall(arguments='{"to":"ilan@example.com","subject":"Hello!","body":"Hi Ilan!"}', call_id='call_AcpCbVCKokzCv8NMP8ZUaUBW', name='send_email', type='function_call', id='fc_6819a3c12ff4819193a73c0d68035a2e0de88d257e540096', status='completed'), ResponseFunctionToolCall(arguments='{"to":"katia@example.com","subject":"Hello!","body":"Hi Katia!"}', call_id='call_msBJbhAgfV0X87gGRA23rCO8', name='send_email', type='function_call', id='fc_6819a3c17a7081919528ed0d178cdda50de88d257e540096', status='completed')]


In [29]:
print(response.output[0].model_dump_json(indent=2))

{
  "arguments": "{\"to\":\"ilan@example.com\",\"subject\":\"Hello!\",\"body\":\"Hi Ilan!\"}",
  "call_id": "call_AcpCbVCKokzCv8NMP8ZUaUBW",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_6819a3c12ff4819193a73c0d68035a2e0de88d257e540096",
  "status": "completed"
}


In [27]:
print(response.output[1].model_dump_json(indent=2))

{
  "arguments": "{\"to\":\"katia@example.com\",\"subject\":\"Hello!\",\"body\":\"Hi Katia!\"}",
  "call_id": "call_msBJbhAgfV0X87gGRA23rCO8",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_6819a3c17a7081919528ed0d178cdda50de88d257e540096",
  "status": "completed"
}


In [30]:
print(response.output[2].model_dump_json(indent=2))

IndexError: list index out of range